<a href="https://colab.research.google.com/github/aswinaus/LLM_Inference/blob/main/rag_Mistral_small_quant_awq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers torch accelerate langchain langchain_huggingface

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-p3ag213v
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-p3ag213v
  Resolved https://github.com/huggingface/transformers to commit 0de15c988b0d27758ce360adb2627e9ea99e91b3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10680508 sha256=3db6438f5f0fb1212053954b59fb60bb3c34b29c5e74102d8fbc5734a8b5a58d
  Stored in directory: /tmp/pip-ephem-wheel-cache-1mk7usf4/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's dependency resolver does n

Code is essentially forcing Python to always use "UTF-8" as the preferred encoding, regardless of the user's actual system settings. UTF-8 is a widely used encoding that can represent a vast range of characters from different languages. By enforcing UTF-8, you can help ensure that your code works consistently across different platforms and avoids encoding-related errors. It's a common practice for improving compatibility and preventing issues with text handling in Python programs.

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
from google.colab import userdata
HUGGING_FACE_TOKEN = userdata.get('HUGGING_FACE_TOKEN')

In [4]:
!huggingface-cli login --token $HUGGING_FACE_TOKEN


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Agents` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Agents`


In [5]:
from google.colab import drive
drive.mount('/content/drive')
# Download Data
data_dir = '/content/drive/MyDrive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from threading import Thread

device = 'cuda' if torch.cuda.is_available() else 'cpu'

The nvidia-smi command is a utility provided by NVIDIA to query and display information about your NVIDIA GPU(s) (Graphics Processing Unit). This includes things like:

GPU model and name
Driver version
GPU utilization
Memory usage
Temperature
Power consumption
Processes running on the GPU

In [7]:
!nvidia-smi

Wed Feb  5 22:55:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             43W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [8]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [9]:
!pip install autoawq

  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.47.1-py3-none-any.whl (10.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0.dev0
    Uninstalling transformers-4.49.0.dev0:
      Successfully uninstalled transformers-4.49.0.dev0


In [1]:
from typing import Tuple, Optional, Union, Dict, Any
from transformers import PreTrainedModel, AutoModel, AutoTokenizer, AutoConfig
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'mistralai/Mistral-Small-24B-Instruct-2501'
quant_path = 'Mistral-Small-24B-Instruct-2501'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 8, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

print(f'Model is quantized and saved at "{quant_path}"')

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

SYSTEM_PROMPT.txt:   0%|          | 0.00/997 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

...

params.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/21.3k [00:00<?, ?B/s]

tekken.json:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200k [00:00<?, ?B/s]